In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import skimage
from skimage import metrics
import PIL

def spatial_kernel(ks, sig_s):
    x = np.linspace(-(ks-1)/2.,(ks-1)/2.0,ks)
    x = x.reshape(len(x), 1)
    x = np.exp(-0.5 * np.square(x) / np.square(sig_s))
    kernel = np.outer(x,x)
    return kernel / np.sum(kernel)
def rtbf(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    w,h,c = image.shape
    I = image.astype(np.float32)
    k_N=18;ks=7;sig_s=6;sig_i=60;
  
    #k_N=12;ks=5;sig_s=5;sig_i=35; # rome parameters for good speed 
    #k_N=21;ks=5;sig_s=10;sig_i=80; # rome parameters for good accuracy
    #k_N=7;ks=3;sig_s=4;sig_i=20; # iitk parameters for good speed
    #k_N=15;ks=5;sig_s=5;sig_i=70; # iitk parameters for good accuracy
    K_array = np.linspace(0, 1, num=k_N).astype(np.float32)

    Wk = np.zeros((w,h,c,k_N),dtype=np.float32)
    Jk = np.zeros((w,h,c,k_N),dtype=np.float32)
    JB = np.zeros((w,h,c,k_N),dtype=np.float32)
    jbK = np.zeros((I.shape),dtype=np.float32)
    jbK1 = np.zeros((I.shape),dtype=np.float32)
    #LK = np.zeros((I.shape),dtype=np.float32)
    #LK1 = np.zeros((I.shape),dtype=np.float32)
    Ib = np.zeros((I.shape),dtype=np.int8)

    Spatial_Kernel = spatial_kernel(ks,sig_s)

    for k in range(k_N-1):
        mul = 1/(np.sqrt(2*np.pi) * sig_i)
        Wk[:,:,:,k] = mul * np.exp(-0.5 * np.square(K_array[k]-I / sig_i))
        Jk[:,:,:,k] = Wk[:,:,:,k]*I
      
        Num_term= cv2.filter2D(src=Jk[:,:,:,k], ddepth=-1, kernel=Spatial_Kernel)
        Den_term= cv2.filter2D(src=Wk[:,:,:,k], ddepth=-1, kernel=Spatial_Kernel)
        JB[:,:,:,k] = np.divide(Num_term,Den_term,out=np.zeros_like(Num_term), where=Den_term!=0)
        
        jbK[:,:,:] = np.where((I[:,:,:]>=K_array[k]) & (I[:,:,:]<K_array[k+1]), K_array[k], JB[:,:,:,k])
        jbK1[:,:,:] = np.where((I[:,:,:]>K_array[k]) & (I[:,:,:]<=K_array[k+1]), K_array[k+1], JB[:,:,:,k])
        LK = (I - K_array[k]) / (K_array[k+1] - K_array[k])
        #Ib = np.uint8(255*(1-(I - LK)*jbK + (I - LK)*jbK1))
        #LK[:,:,:] = np.where((I>K_array[k]) & (I<K_array[k+1]), K_array[k], I)
        #LK1[:,:,:] = np.where((I>K_array[k]) & (I<=K_array[k+1]), K_array[k+1], I)
    #Ib = np.uint8(255*((LK1*100 - I)*jbK*100 + (I - LK*100)*jbK1*100))
    #I'(x) = (1 - q) * J(k[p],x) + q * J(k[p+1],x)
    Ib = ((1-LK)*jbK + LK*jbK1).astype(dtype=np.uint8)  
    Ib = cv2.cvtColor(Ib, cv2.COLOR_RGB2BGR)
    return(Ib)
def metrics(img,img_filter):
    psnr=skimage.metrics.peak_signal_noise_ratio(img, img_filter, data_range=None)
    ssim=skimage.metrics.structural_similarity(img, img_filter, win_size=None, gradient=False, data_range=None, channel_axis=None, multichannel=True, gaussian_weights=False, full=False)
    return [psnr,ssim]

img1 = cv2.imread('rome.jpg')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.imread('iitk.jpg')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

start1 = time.time()
rtbf_img1 = rtbf(img1)
cv2.imwrite('rtbf_img_rome.jpg', rtbf_img1, [int(cv2.IMWRITE_JPEG_QUALITY), 99])
rback1 = cv2.imread('rtbf_img_rome.jpg')
[psnr1,ssim1]=metrics(rback1,rtbf_img1)
execution_time1=time.time()-start1

start2 = time.time()
rtbf_img2 = rtbf(img2)
cv2.imwrite('rtbf_img_iitk.jpg', rtbf_img2, [int(cv2.IMWRITE_JPEG_QUALITY), 99])
rback2 = cv2.imread('rtbf_img_iitk.jpg')
[psnr1,ssim1]=metrics(rback2,rtbf_img2)
execution_time2=time.time()-start2

print("****************************************************************")
print('PSNR and SSIM for rome:',psnr1 ,'and', ssim1)
print("Runtime for rome: " +str(execution_time1),"seconds")
print("----------------------------------------------------------------")
print('PSNR and SSIM for iitk:',psnr2 ,'and', ssim2)
print("Runtime for iitk: " +str(execution_time2),"seconds")
print("****************************************************************")
plt.figure(figsize=[15,15], dpi=100)
plt.subplot(2, 2, 1)
plt.imshow(img1); plt.title("Rome original")
plt.subplot(2, 2, 2)
plt.imshow(rtbf_img1); plt.title("Rome RTBF")
plt.subplot(2, 2, 3)
plt.imshow(img2); plt.title("IITK original")
plt.subplot(2, 2, 4)
plt.imshow(rtbf_img2); plt.title("IITK RTBF")
plt.tight_layout()


Output hidden; open in https://colab.research.google.com to view.